### Imports

In [23]:
import json
from pathlib import Path
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

### Load input data

In [24]:
data_path = Path("data") / "technology_data.json"

with data_path.open("r", encoding="utf-8") as f:
    technology_data = json.load(f)

TECHNOLOGY_UNITS = technology_data["TECHNOLOGY_UNITS"]  
TECHNOLOGY_DATA = technology_data["TECHNOLOGY_DATA"]

df_units = pd.DataFrame(list(TECHNOLOGY_UNITS.items()), columns=["Parameter", "Unit"])
df_tech = pd.DataFrame(TECHNOLOGY_DATA).reset_index().rename(columns={"index": "parameter"})
df = df_units.merge(df_tech, left_on="Parameter", right_on="parameter", how="right").drop(columns=["parameter"])
df

,Parameter,Unit,Gas turbine (simple cycle),Natural gas engine plant,Diesel engine farm,OCGT - Natural gas,Coal power plant,Nuclear power plant,Onshore wind,Offshore wind (fixed),Utility-scale PV
0,nominal_investment_total,MEUR/MW_el,0.600,0.50,0.36,0.47,2.10,4.000,1.150,2.390,0.38
1,fixed_om_total,EUR/MW/year,19778.730,6646.08,8983.37,8236.12,34324.40,147700.000,16663.000,34000.000,9500.00
2,variable_om_total,EUR/MWh,4.470,6.38,6.38,4.79,3.21,20.000,1.980,3.450,0.50
3,elec_eff,fraction (0-1),0.410,0.48,0.35,0.41,0.52,0.360,0.410,0.520,0.16
4,technical_lifetime,years,25.000,25.00,25.00,25.00,25.00,60.000,30.000,30.000,35.00
5,construction_time,years,1.500,1.00,1.00,0.20,4.50,8.000,1.500,3.500,0.50
6,total_outage,fraction of time (0-1),0.025,0.01,0.01,0.01,0.00,0.001,0.023,0.025,NaN


In [25]:
# Define the initial budget (in MEUR)
initial_budget = 500  # The total available budget for investment in MW

# Assumed energy demand (in MWh), will not be used for meeting demand directly in this case
# Instead, we focus on energy mix selection based on budget and cost minimization
total_energy_mix = 1000000  # in MWh

# Define a large constant for big-M method
M = 1000000  # Large number to enforce energy production only when selected

# Hours in the year
hours_per_year = 8760

In [29]:
# Initialize the model
model = gp.Model("Optimal_Plant_Mix")

# Define decision variables for each technology
tech_names = list(TECHNOLOGY_DATA.keys())
investment_vars = model.addVars(tech_names, vtype=GRB.CONTINUOUS, name="Investment")

# Total installed capacity for each technology in MW (decision variable)
energy_produced_vars = model.addVars(tech_names, vtype=GRB.CONTINUOUS, name="EnergyProduced")

# Define binary variables to track if a technology is producing energy
binary_vars = model.addVars(tech_names, vtype=GRB.BINARY, name="BinaryProduced")

# Objective: Minimize the total annualized cost (CAPEX + O&M)
model.setObjective(
    gp.quicksum(
        investment_vars[tech] * (TECHNOLOGY_DATA[tech]['nominal_investment_total']/TECHNOLOGY_DATA[tech]['technical_lifetime']) for tech in tech_names
    ) +  # CAPEX cost in MEUR  --  devided by lifetime to annualize
    gp.quicksum(
        energy_produced_vars[tech] * (TECHNOLOGY_DATA[tech]['variable_om_total']/1000000) for tech in tech_names
    ),  # Variable O&M cost in MEUR
    GRB.MINIMIZE
)

# Constraints:

# 1. Ensure total annualized CAPEX stays within the initial budget
model.addConstr(
    gp.quicksum(
        investment_vars[tech] * TECHNOLOGY_DATA[tech]['nominal_investment_total'] for tech in tech_names
    ) <= initial_budget,  # Total investment must not exceed budget
    "Budget_Constraint"
)

# 2. Ensure the total energy produced by the selected technologies equals the required energy mix
model.addConstr(
    gp.quicksum(
        energy_produced_vars[tech] for tech in tech_names
    ) == total_energy_mix,  # Total energy produced should match the target energy mix
    "EnergyMix_Constraint"
)

# 3. Ensure at least 4 different technologies are selected
# Sum of binary variables should be at least 4
model.addConstr(
    gp.quicksum(
        binary_vars[tech] for tech in tech_names
    ) >= 4,  # At least 4 technologies must be producing energy
    "MinTechnologies_Constraint"
)

# 4. Ensure each selected technology produces at least 10% of the total energy mix
# Big-M method to link the energy production with the binary variable
model.addConstrs(
    (energy_produced_vars[tech] >= 0.1 * total_energy_mix * binary_vars[tech] for tech in tech_names)
    , name="MinTechnologyContribution"
)

# 5. The energy produced by each technology cannot exceed its installed capacity
model.addConstrs(
    (energy_produced_vars[tech] <= investment_vars[tech] * hours_per_year * TECHNOLOGY_DATA[tech]['elec_eff'] for tech in tech_names)
    , name="MaxCapacity_Constraint"
)

# 6. Link binary variable with energy production (a binary variable is 1 if energy is produced, 0 otherwise)
# Using big-M to activate energy production only when binary variable is 1
model.addConstrs(
    (energy_produced_vars[tech] <= M * binary_vars[tech] for tech in tech_names)
    , name="BinaryLink"
)

# Solve the model
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 30 rows, 27 columns and 81 nonzeros
Model fingerprint: 0xa7a592c6
Variable types: 18 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [4e-01, 1e+06]
  Objective range  [5e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 1e+06]
Presolve removed 9 rows and 9 columns
Presolve time: 0.01s
Presolved: 21 rows, 18 columns, 63 nonzeros
Variable types: 9 continuous, 9 integer (9 binary)
Found heuristic solution: objective 10.3536452

Root relaxation: objective 8.997711e+00, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       8.9977112    8.99771  0.00%     -    

In [31]:
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    print(f"Total annual cost: {model.objVal:.2f} MEUR")
    print("Optimal plant mix (MW) and energy produced (MWh):")
    for tech in tech_names:
        if energy_produced_vars[tech].x > 0:  # Only print technologies that contribute to the energy mix
            print(f"{tech}: {investment_vars[tech].x:.4f} MW installed, {energy_produced_vars[tech].x:.2f} MWh produced, covering {(100*energy_produced_vars[tech].x/total_energy_mix):.2f}% of the demand")
else:
    print("Optimization did not succeed:", model.status)


Optimal solution found:
Total annual cost: 9.00 MEUR
Optimal plant mix (MW) and energy produced (MWh):
Gas turbine (simple cycle): 27.8427 MW installed, 100000.00 MWh produced, covering 10.00% of the demand
Diesel engine farm: 32.6158 MW installed, 100000.00 MWh produced, covering 10.00% of the demand
OCGT - Natural gas: 27.8427 MW installed, 100000.00 MWh produced, covering 10.00% of the demand
Utility-scale PV: 499.4292 MW installed, 700000.00 MWh produced, covering 70.00% of the demand


In [32]:
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    print(f"Total annual cost: {model.objVal:.2f} MEUR")
    print("\nDetailed cost per technology:")
    print("-------------------------------------------------------------")

    for tech in tech_names:
        cap = investment_vars[tech].x
        gen = energy_produced_vars[tech].x

        if gen > 0:
            capex_total = cap * TECHNOLOGY_DATA[tech]['nominal_investment_total']
            capex_annual = capex_total / TECHNOLOGY_DATA[tech]['technical_lifetime']
            vom_annual = gen * TECHNOLOGY_DATA[tech]['variable_om_total'] / 1_000_000

            print(f"{tech}")
            print(f"  Installed capacity: {cap:.2f} MW")
            print(f"  Energy produced:    {gen:,.0f} MWh/year")
            print(f"  Total CAPEX:        {capex_total:.2f} MEUR")
            print(f"  Annual CAPEX:       {capex_annual:.2f} MEUR/year")
            print(f"  Variable O&M:       {vom_annual:.2f} MEUR/year")
            print(f"  Share of demand:    {100*gen/total_energy_mix:.2f}%")
            print("-------------------------------------------------------------")

Optimal solution found:
Total annual cost: 9.00 MEUR

Detailed cost per technology:
-------------------------------------------------------------
Gas turbine (simple cycle)
  Installed capacity: 27.84 MW
  Energy produced:    100,000 MWh/year
  Total CAPEX:        16.71 MEUR
  Annual CAPEX:       0.67 MEUR/year
  Variable O&M:       0.45 MEUR/year
  Share of demand:    10.00%
-------------------------------------------------------------
Diesel engine farm
  Installed capacity: 32.62 MW
  Energy produced:    100,000 MWh/year
  Total CAPEX:        11.74 MEUR
  Annual CAPEX:       0.47 MEUR/year
  Variable O&M:       0.64 MEUR/year
  Share of demand:    10.00%
-------------------------------------------------------------
OCGT - Natural gas
  Installed capacity: 27.84 MW
  Energy produced:    100,000 MWh/year
  Total CAPEX:        13.09 MEUR
  Annual CAPEX:       0.52 MEUR/year
  Variable O&M:       0.48 MEUR/year
  Share of demand:    10.00%
-----------------------------------------------